In [ ]:
!pip install mediapipe

In [ ]:
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

In [ ]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

def extract_info_from_path(path):
    parts = path.split(os.sep)
    main_category = parts[-3]
    sub_category = parts[-2]
    image_name = parts[-1].split('_visualization')[0]
    return main_category, sub_category, image_name

def extract_heatmap(image_path):
    img = Image.open(image_path)
    width, height = img.size
    plot_width = width // 3
    heatmap = img.crop((plot_width * 2, 0, width, height))
    if heatmap.mode == 'RGBA':
        heatmap = heatmap.convert('RGB')
    return heatmap

def draw_landmarks_on_image(rgb_image, detection_result):
    face_landmarks_list = detection_result.face_landmarks
    annotated_image = np.copy(rgb_image)

    for face_landmarks in face_landmarks_list:
        face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        face_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
        ])

        mp.solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style())
        mp.solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_contours_style())
        mp.solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_iris_connections_style())

    return annotated_image

def analyze_focus_areas(detection_result, heatmap):
    if detection_result.face_landmarks:
        face_landmarks = detection_result.face_landmarks[0]
        focus_scores = {}
        height, width = heatmap.shape[:2]
        heatmap_red = heatmap[:,:,2] if len(heatmap.shape) == 3 else heatmap

        for feature, indices in features.items():
            feature_mask = np.zeros((height, width), dtype=np.uint8)
            points = []
            for idx in indices:
                pt = face_landmarks[idx]
                x, y = int(pt.x * width), int(pt.y * height)
                if 0 <= x < width and 0 <= y < height:
                    points.append([x, y])

            if len(points) > 2:
                hull = cv2.convexHull(np.array(points))
                cv2.fillConvexPoly(feature_mask, hull, 255)
                kernel = np.ones((3,3), np.uint8)
                feature_mask = cv2.dilate(feature_mask, kernel, iterations=1)
                mean_intensity = cv2.mean(heatmap_red, mask=feature_mask)[0]
                focus_scores[feature] = mean_intensity
            else:
                focus_scores[feature] = 0

        min_score = min(focus_scores.values())
        max_score = max(focus_scores.values())
        if max_score > min_score:
            focus_scores = {k: (v - min_score) / (max_score - min_score) for k, v in focus_scores.items()}
        else:
            focus_scores = {k: 1.0 if v == max_score else 0.0 for k, v in focus_scores.items()}

        return focus_scores
    return None

def process_image(image_path, subfolder_name):
    try:
        heatmap = extract_heatmap(image_path)
        if heatmap is None:
            return {
                'Main Category': 'white_females',
                'Subfolder Name': subfolder_name,
                'Image Name': os.path.basename(image_path),
                'Focus Scores': None,
                'Error': 'Failed to extract heatmap'
            }

        heatmap_array = np.array(heatmap)
        heatmap_image_rgb = cv2.cvtColor(heatmap_array, cv2.COLOR_RGB2BGR)

        base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
        options = vision.FaceLandmarkerOptions(base_options=base_options, num_faces=1)
        detector = vision.FaceLandmarker.create_from_options(options)

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=heatmap_image_rgb)
        detection_result = detector.detect(mp_image)

        if not detection_result.face_landmarks:
            return {
                'Main Category': 'white_females',
                'Subfolder Name': subfolder_name,
                'Image Name': os.path.basename(image_path),
                'Focus Scores': None,
                'Error': 'No face detected'
            }

        focus_scores = analyze_focus_areas(detection_result, heatmap_image_rgb)

        return {
            'Main Category': 'white_females',
            'Subfolder Name': subfolder_name,
            'Image Name': os.path.basename(image_path),
            'Focus Scores': focus_scores,
            'Error': None
        }
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return {
            'Main Category': 'white_females',
            'Subfolder Name': subfolder_name,
            'Image Name': os.path.basename(image_path),
            'Focus Scores': None,
            'Error': str(e)
        }

def process_directory(directory):
    results = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".png"):
                image_path = os.path.join(root, file)
                main_category, sub_category, image_name = extract_info_from_path(image_path)
                focus_scores = process_image(image_path)
                results.append((main_category, sub_category, image_name, focus_scores))
    return results


def process_folder(folder_path, subfolder_name):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            result = process_image(image_path, subfolder_name)
            results.append(result)
    return results


def save_results_to_csv(results, output_file):
    all_features = set()
    for result in results:
        if result['Focus Scores']:
            all_features.update(result['Focus Scores'].keys())
    all_features = sorted(list(all_features))
    columns = ['Main Category', 'Subfolder Name', 'Image Name', 'Highest Focus Feature', 'Error']
    top_k = 5
    for i in range(1, top_k + 1):
        columns.append(f'Top-{i} Feature')
        columns.append(f'Top-{i} Score')

    columns.extend(all_features)
    data = []
    for result in results:
        row = {
            'Main Category': result['Main Category'],
            'Subfolder Name': result['Subfolder Name'],
            'Image Name': result['Image Name'],
            'Error': result.get('Error', '')
        }

        if result['Focus Scores']:
            sorted_scores = sorted(result['Focus Scores'].items(), key=lambda item: item[1], reverse=True)
            row['Highest Focus Feature'] = sorted_scores[0][0]
            for i in range(top_k):
                if i < len(sorted_scores):
                    feature, score = sorted_scores[i]
                    row[f'Top-{i+1} Feature'] = feature
                    row[f'Top-{i+1} Score'] = score
                else:
                    row[f'Top-{i+1} Feature'] = ''
                    row[f'Top-{i+1} Score'] = ''
            for feature in all_features:
                row[feature] = result['Focus Scores'].get(feature, '')
        else:
            row['Highest Focus Feature'] = ''
            for i in range(top_k):
                row[f'Top-{i+1} Feature'] = ''
                row[f'Top-{i+1} Score'] = ''
            for feature in all_features:
                row[feature] = ''

        data.append(row)

    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")


def process_main_directory(main_directory):
    all_results = []
    subdirs = [d for d in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, d))]

    with tqdm(total=len(subdirs), desc="Processing folders") as pbar:
        for subfolder_name in subdirs:
            folder_path = os.path.join(main_directory, subfolder_name)
            print(f"\nProcessing folder: {subfolder_name}")
            results = process_folder(folder_path, subfolder_name)
            all_results.extend(results)
            pbar.update(1)

    return all_results


if __name__ == "__main__":
    main_directory = '/content/white_females_visualization'
    results = process_main_directory(main_directory)
    output_file = os.path.join(main_directory, 'white_females_focus_scores.csv')
    save_results_to_csv(results, output_file)



